# Kashgari NER Benchmarks

- Kashgari: 2.1.0
- TensorFLow: 2.1.0

Download Embeddings to Embddings Folder and unzip.
- [BERT-Base, Chinese](https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip)

Final folder struct is

```
.
└── embeddings
    └── chinese_L-12_H-768_A-12
```

In [ ]:
RUNNING_ON_CLOUD = False

if RUNNING_ON_CLOUD:
    !mkdir embeddings
    !wget https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip embeddings/chinese_L-12_H-768_A-12.zip
    !unzip embeddings/chinese_L-12_H-768_A-12.zip

In [1]:
# Setup macros
if RUNNING_ON_CLOUD:
    EMBEDDING_FOLDER = './embeddings'
    TF_LOG_FOLDER = './tf_dir'
    LOG_FILE_PATH = './ner_training_log.json'
else:
    EMBEDDING_FOLDER = '/Users/brikerman/Desktop/kashgari-demo/embeddings'
    TF_LOG_FOLDER = 'tf_dir'
    LOG_FILE_PATH = 'ner_training_log.json'

In [2]:
from kashgari.corpus import ChineseDailyNerCorpus

train_x, train_y = ChineseDailyNerCorpus.load_data('train')
test_x, test_y = ChineseDailyNerCorpus.load_data('test')
valid_x, valid_y = ChineseDailyNerCorpus.load_data('valid')

train_x = train_x[:200]
train_y = train_y[:200]

In [3]:
import os
import json
from tensorflow import keras
from kashgari.tasks.labeling import BiLSTM_Model, BiLSTM_CRF_Model
from kashgari.tasks.labeling import BiGRU_Model, BiGRU_CRF_Model
from kashgari.callbacks import EvalCallBack

from kashgari.embeddings import WordEmbedding, BertEmbedding
from IPython.display import clear_output

In [4]:
bert_chinese = BertEmbedding(os.path.join(EMBEDDING_FOLDER, 'chinese_L-12_H-768_A-12'))
embeddings = [
    # ('Bert-Chinese', bert_chinese),
    ('Bare', None)
]

model_classes = [
    ('BiLSTM', BiLSTM_Model), 
    ('BiLSTM_CRF', BiLSTM_CRF_Model), 
    ('BiGRU', BiGRU_Model), 
    ('BiGRU_CRF', BiGRU_CRF_Model)
]

In [7]:
for embed_name, embed in embeddings:
    for model_name, MOEDL_CLASS in model_classes:
        run_name = f"{embed_name}-{model_name}"
        model = MOEDL_CLASS(embed)
        
        early_stop = keras.callbacks.EarlyStopping(patience=5)
        reduse_lr_callback = keras.callbacks.ReduceLROnPlateau(factor=0.1, 
                                                               patience=5)

        eval_callback = EvalCallBack(task_model=model,
                                     valid_x=valid_x, 
                                     valid_y=valid_y,
                                     step=1)

        tf_board = keras.callbacks.TensorBoard(
            log_dir=os.path.join(TF_LOG_FOLDER, run_name), 
            update_freq=1000
        )

        callbacks = [early_stop, reduse_lr_callback, eval_callback, tf_board]

        model.fit(train_x, 
                  train_y,
                  valid_x,
                  valid_y,
                  callbacks=callbacks,
                  epochs=1)

        if os.path.exists(LOG_FILE_PATH):
            logs = json.load(open(LOG_FILE_PATH, 'r'))
        else:
            logs = {}
        
        logs[run_name] = eval_callback.logs
        
        with open(LOG_FILE_PATH, 'w') as f:
            f.write(json.dumps(logs, indent=2))

Calculating sequence length: 100%|██████████| 200/200 [00:00<00:00, 584979.64it/s]
Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, None)]            0         
_________________________________________________________________
layer_embedding (Embedding)  (None, None, 100)         69900     
_________________________________________________________________
layer_blstm (Bidirectional)  (None, None, 256)         234496    
_________________________________________________________________
layer_dropout (Dropout)      (None, None, 256)         0         
_________________________________________________________________
dense_6 (Dense)              (None, None, 8)           2056      
_________________________________________________________________
activation_4 (Activation)    (None, None, 8)           0         
Total params: 306,452
Trainable params: 3

In [ ]:
model.evaluate(train_x[:200], train_y[:200])

In [ ]:
model.predict(train_x[:200])